# `ipywidgets` Interactive Demo

Simple demonstration of rendering Interactive widgets in a `jupyterlite` notebook.

`ipywidgets` can be installed in this deployment (it provides the @jupyter-widgets/jupyterlab-manager federated extension), but you will need to make your own deployment to have access to other interactive widgets libraries.

In [ ]:
import piplite

await piplite.install('ipywidgets')

In [ ]:
from ipywidgets import IntSlider

In [ ]:
slider = IntSlider()

In [ ]:
slider

In [ ]:
slider

In [ ]:
slider.value

In [ ]:
slider.value = 5

In [ ]:
from ipywidgets import IntText, link

In [ ]:
text = IntText()

In [ ]:
text

In [ ]:
link((slider, 'value'), (text, 'value'));

In [ ]:
from ipywidgets import interact
from IPython.display import Markdown, display

@interact(cookies=slider)
def cookies(cookies=slider.value, calories=(0,150)):
    total_calories = calories * cookies
    if cookies:
        display(Markdown(f"If each cookie contains _{calories} calories_, _{cookies} cookies_ contain **{total_calories} calories**!"))
    else:
        display(Markdown(f"No cookies!"))
    if total_calories > 2000:
        display(Markdown(f"> Maybe that's too many cookies..."))
